In [6]:
#import
import math
import torch
import torchmetrics

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import lightning.pytorch as pl
import torch.nn.functional as F

from sklearn.metrics import mean_squared_error
from lightning.pytorch.loggers import CSVLogger
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.loggers import WandbLogger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

In [7]:
#make screen wide
from IPython.core.display import display, HTML
from IPython import get_ipython as get_ipython
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important;margin-left:-30px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
    '</style>'
))

# FCNN

In [8]:
logger_path = './csv_logs'
data_path = '../data/clasdb_pi_plus_n.txt'

name_of_the_experiment = 'first'
logger_version = '1.0015'


scale_data = False
test_size = 0.1
batch_size = 16
net_architecture = [5, 60, 80, 100, 120, 140, 140, 140, 140, 140, 140, 140, 140, 140, 100, 80, 60, 20, 1]
lr = 0.00005
activation_function = nn.ReLU()
loss_func = RMSELoss()
optim_func = 'ADAM'
max_epochs = 2000
min_delta = 0.0001
patience = 20

hyperparams_dict = {
    'scale_data': scale_data,
    'test_size': test_size,
    'batch_size': batch_size,
    'lr': lr,
    'net_architecture': net_architecture,
    'activation_function': activation_function,
    'loss_func': loss_func,
    'optim_func': optim_func,
    'max_epochs': max_epochs,
    'min_delta': min_delta,
    'patience': patience
}

In [9]:
class InterpolDataSet(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        self.len = len(labels)

    def __getitem__(self, index):
        feature = self.features[index]
        label = self.labels[index]
        return feature, label

    def __len__(self):
        return self.len

class InterpolDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.df = None
    
    def setup(self, stage):
        # data reading and preprocessing
        df = pd.read_csv(data_path, delimiter='\t', header=None)
        df.columns = ['Ebeam', 'W', 'Q2', 'cos_theta', 'phi', 'dsigma_dOmega', 'error', 'id']

        df.loc[8314:65671, 'Ebeam'] = 5.754 # peculiarity of this dataset.
        df['phi'] = df.phi.apply(lambda x: math.radians(x))
        df = df.drop('id', axis=1)
        df = df.iloc[df[['Ebeam', 'W', 'Q2', 'cos_theta', 'phi']].drop_duplicates().index]
        self.df = df
        
        #train test split
        feature_data = df[['Ebeam', 'W', 'Q2', 'cos_theta', 'phi']]
        label_data = df['dsigma_dOmega']
        
        if scale_data:
            scaler_feature = StandardScaler()
            scaler_target = StandardScaler()
            feature_data = scaler_feature.fit_transform(feature_data)
            label_data = scaler_target.fit_transform(label_data.values.reshape(-1,1))
        else:
            pass
        
        train_feature_data, test_feature_data, train_label_data, test_label_data = train_test_split(feature_data, label_data, 
                                                                                                    test_size=test_size, random_state=1438)
        
        
        self.train_dataset = InterpolDataSet(torch.tensor(train_feature_data.values, dtype=torch.float32), 
                                             torch.tensor(train_label_data.values, dtype=torch.float32))
        self.test_dataset = InterpolDataSet(torch.tensor(test_feature_data.values, dtype=torch.float32), 
                                            torch.tensor(test_label_data.values, dtype=torch.float32))

    def train_dataloader(self):
        return DataLoader(dataset = self.train_dataset, batch_size = batch_size, shuffle = False, num_workers=0)

    def val_dataloader(self):
        return DataLoader(dataset = self.test_dataset, batch_size = batch_size, shuffle = False, num_workers=0)
    
class RMSELoss(torch.nn.Module):
    def __init__(self):
        super(RMSELoss,self).__init__()

    def forward(self,x,y):
        criterion = nn.MSELoss()
        loss = torch.sqrt(criterion(x, y))
        return loss

In [12]:
class InterpolRegressor(pl.LightningModule):
    def __init__(self, hyperparams):
        super(InterpolRegressor, self).__init__()
        
        self.hyperparams = hyperparams
        self.loss_func = self.hyperparams.get('loss_func')
        self.net_architecture = self.hyperparams.get('net_architecture')
        self.activation_function = self.hyperparams.get('activation_function')
        self.net = nn.Sequential()
        self.train_loss = 0
        self.val_loss = 0
                
        for i in range(1,len(self.net_architecture)):
            self.net.append(nn.Linear(self.net_architecture[i-1], self.net_architecture[i]))
            if i!=len(self.net_architecture)-1:
                self.net.append(self.activation_function)
            else:
                pass
            
        self.save_hyperparameters()
        logger.log_hyperparams(self.hyperparams)
        

    def forward(self, x):
        return self.net(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_func
        self.train_loss = loss(y_hat.reshape(-1), y)
        self.log('train_loss', self.train_loss, on_step=False, on_epoch=True, prog_bar=False)
        return self.train_loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_func
        self.val_loss = loss(y_hat.reshape(-1), y)
        self.log('val_loss', self.val_loss, on_step=False, on_epoch=True, prog_bar=False)
        return self.val_loss
    
#     def on_train_epoch_end(self):
#         self.log('train_loss', self.train_loss, on_step=False, on_epoch=True, prog_bar=False)

#     def on_validation_epoch_end(self):
#         self.log('step', self.trainer.current_epoch)
#         self.log('val_loss', self.val_loss, on_step=False, on_epoch=True, prog_bar=False)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

In [ ]:
wandb

In [13]:
data_module = InterpolDataModule()
logger = CSVLogger(save_dir=logger_path, 
                   name=name_of_the_experiment,
                   version=logger_version)
model = InterpolRegressor(hyperparams=hyperparams_dict)

early_stop_callback = [EarlyStopping(monitor="val_loss", mode="min", min_delta=min_delta, patience=patience, verbose=True)]
trainer = pl.Trainer(max_epochs=max_epochs,
                     accelerator='cpu',
                     callbacks=early_stop_callback,
                     logger=logger,
                     enable_progress_bar=False,
                     val_check_interval = 1)
trainer.fit(model, data_module)

/Users/andrey.golda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/lightning/fabric/loggers/csv_logs.py:198: Experiment logs directory ./csv_logs/first/1.0015 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1)` was configured so validation will run after every batch.
/Users/andrey.golda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/lightning/pytorch/utilities/parsing.py:43: attribute 'hyperparams' removed from hparams because it cannot be pickled
/Users/andrey.golda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:639: Checkpoint directory ./csv_logs/first/1.0015/checkpoints exists and is not empty.

  | Name                | Type       | Params
---------------------------------------------------
0 | 

In [23]:
preds = model.forward(data_module.train_dataset.features)
mean_squared_error(data_module.train_dataset.labels, preds.detach().numpy())

1.0293056

In [24]:
preds = model.forward(data_module.test_dataset.features)
mean_squared_error(data_module.test_dataset.labels, preds.detach().numpy())

0.9479398

In [22]:
# model = InterpolRegressor.load_from_checkpoint('./csv_logs/first/1.0002/checkpoints/epoch=9-step=52560.ckpt')
# model.eval()

df = pd.read_csv(f'./csv_logs/first/{logger_version}/metrics.csv')
df

,val_loss,step,epoch,train_loss
0,0.780708,5255,0,NaN
1,NaN,5255,0,0.813510
2,0.780709,10511,1,NaN
3,NaN,10511,1,0.813444
4,0.780724,15767,2,NaN
5,NaN,15767,2,0.813409
6,0.780734,21023,3,NaN
7,NaN,21023,3,0.813390
8,0.780734,26279,4,NaN
9,NaN,26279,4,0.813384
